<a href="https://colab.research.google.com/github/debaggi/DTCW-BZ-DA-ML-DS/blob/main/Crowdlend_case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [4]:
loan_data = pd.read_excel('/content/drive/MyDrive/Crowdlend | Data Analysis Project/loan_table.xlsx')
investor_cashflow = pd.read_excel('/content/drive/MyDrive/Crowdlend | Data Analysis Project/investor_cashflow.xlsx')

In [5]:
# вывожу сведения об атрибутах таблицы investor_ cashflow
investor_cashflow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21093 entries, 0 to 21092
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   investor_id        21093 non-null  int64         
 1   date               21093 non-null  datetime64[ns]
 2   loan_id            21093 non-null  int64         
 3   amount             21093 non-null  float64       
 4   principal_payment  21093 non-null  float64       
 5   interest_payment   21093 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(2)
memory usage: 988.9 KB


In [7]:
loan_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   loan_id          1000 non-null   int64         
 1   loan_rate        1000 non-null   float64       
 2   amount           1000 non-null   int64         
 3   term             1000 non-null   int64         
 4   status           1000 non-null   object        
 5   start_date       1000 non-null   datetime64[ns]
 6   date_of_default  76 non-null     datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(3), object(1)
memory usage: 54.8+ KB


**Описание данных**


investor_cashflow
----------------
*   investor_id
*   loan_id
*   principal_payment
*   total_amount
*   interest_payment


loan_data
----------------
*   loan_id
*   loan_rate
*   raised_amount
*   term
*   status
*   start_date
*   date_of_default



In [ ]:
# переимновываю столбцы датафреймов
  # raised_amount: привлеченная [от инвестора] сумма
investor_cashflow.rename(columns={'amount':'total_amount'}, inplace=True)

# transaction_date: дата транзакции от инвестора заемщику или в обратном направлении
loan_data.rename(columns={'date':'transaction_date',
                          'amount':'raised_amount'}, inplace=True)

print(f'Ренейминг столбцов завершен')

# преобразовывую investor_id и loan_id к типу str в датафреймах investor_cashflow и loan_data
investor_cashflow['loan_id'] = investor_cashflow['loan_id'].apply(lambda x: str(x))
loan_data['loan_id'] = loan_data['loan_id'].apply(lambda y: str(y))
investor_cashflow['investor_id'] = investor_cashflow['investor_id'].apply(lambda y: str(y))

print(f'Типы данных успешно изменены')

Ренейминг столбцов завершен
Типы данных успешно изменены


###**1 задание**

Необходимо рассчитать число активных займов в портфелях инвесторов.
В качестве ответа нужно предоставить разницу по модулю этих значений между инвесторами с id = 1 и id = 3.


In [ ]:
# выбираю уникальные идентификаторы активных займов
active_loan_id = np.unique(loan_data[loan_data['status']=='active']['loan_id'].values)

# показываю активные займы инвесторов с идентифкаторами 1 и 3
active_loans_by_select_investors = investor_cashflow[(investor_cashflow.loan_id.isin(active_loan_id)==True) & (investor_cashflow['investor_id'].isin(['1', '3'])==True)][['investor_id','loan_id']].groupby(by='investor_id').agg({'loan_id':'count'})
active_loans_by_select_investors

# вычисляю разность количества активных займов у инвесторов с идентификаторами 1 и 3
delta_acitve_loans = np.abs(active_loans_by_select_investors.iloc[0]['loan_id'] - active_loans_by_select_investors.iloc[1]['loan_id'])
print(f'Разность количества активных займов для инвесторов 1 и 3: {delta_acitve_loans}')

Разность количества активных займов для инвесторов 1 и 3: 317


## **2 задание**

Необходимо рассчитать:

* какую долю от общей суммы каждого займа составляет инвестиция каждого конкретного инвестора.

* Среди рассчитанных значений  получить среднее значение среди всех, а также максимальное значение для займа с id = 3.

> В качестве ответа предоставить разницу по модулю двух полученных выше значений (среднего из всех и максимального по займу с id = 3), умножить на 10 000 и взять от полученного числа целую часть

## **Создаю общую таблицу, которая описывает параметры займов и инвестиций**

> объединяю данные из таблицы investor_cashflow и loan_data по идентификатору займа

In [ ]:
full_inner_data = pd.merge(loan_data, investor_cashflow, how='inner', on='loan_id')
full_inner_data.head(3)

,loan_id,loan_rate,raised_amount,term,status,start_date,date_of_default,investor_id,date,total_amount,principal_payment,interest_payment
0,592,0.395,1500000,360,closed,2022-05-01,NaT,3,2022-05-01,-11000.000000,-11000.000000,0.000000
1,592,0.395,1500000,360,closed,2022-05-01,NaT,3,2022-05-31,1124.410909,762.327576,362.083333
2,592,0.395,1500000,360,closed,2022-05-01,NaT,3,2022-06-30,1124.410909,787.420858,336.990051


In [ ]:
select_data = full_inner_data[['loan_id','term', 'loan_rate', 'investor_id','raised_amount', 'principal_payment']]

In [ ]:
# выбираю записи об инвестициях в займы
select_data = select_data.query('principal_payment < 0')
select_data

,loan_id,term,loan_rate,investor_id,raised_amount,principal_payment
0,592,360,0.395,3,1500000,-11000.0
13,781,180,0.395,1,1500000,-26250.0
14,781,180,0.395,3,1500000,-11000.0
15,781,180,0.395,4,1500000,-2500.0
34,381,360,0.206,2,5000000,-16000.0
...,...,...,...,...,...,...
21034,814,360,0.206,4,1250000,-500.0
21035,814,360,0.206,5,1250000,-10000.0
21063,843,360,0.206,1,1250000,-5250.0
21064,843,360,0.206,2,1250000,-16000.0


In [ ]:
# вычисляю долю инвестиций в суммах займов: PIiLA
# PIiLA = part of investment in loan amount
select_data['PIiLA, %'] = select_data['principal_payment'] / select_data['raised_amount'] * 100

# беру абсолют по вычисленному параметру PIiLA
select_data['PIiLA, %'] = np.abs(select_data['PIiLA, %'])
select_data

,loan_id,term,loan_rate,investor_id,raised_amount,principal_payment,"PIiLA, %"
0,592,360,0.395,3,1500000,-11000.0,0.733333
13,781,180,0.395,1,1500000,-26250.0,1.750000
14,781,180,0.395,3,1500000,-11000.0,0.733333
15,781,180,0.395,4,1500000,-2500.0,0.166667
34,381,360,0.206,2,5000000,-16000.0,0.320000
...,...,...,...,...,...,...,...
21034,814,360,0.206,4,1250000,-500.0,0.040000
21035,814,360,0.206,5,1250000,-10000.0,0.800000
21063,843,360,0.206,1,1250000,-5250.0,0.420000
21064,843,360,0.206,2,1250000,-16000.0,1.280000


In [ ]:
# рассчитываю среднее значение параметра PIiLA
mean_piila = np.round(select_data['PIiLA, %'].mean(), 2)
print(f'Средняя доля инвестиции среди всех займов = {mean_piila}%')

Средняя доля инвестиции среди всех займов = 0.99%


In [ ]:
# нахожу максимальную долю инвестиции в займе с идентификатором loan_id = 3
max_piila_in_loan_id_3 = np.round(select_data.query('loan_id=="3"')['PIiLA, %'].max(), 2)
print(f'Максимальная доля инвестиции в займе с loan_id=3: {max_piila_in_loan_id_3}%')

Максимальная доля инвестиции в займе с loan_id=3: 1.1%


In [ ]:
# Абсолютная величина разности средней доли инвестиции среди всех займов и максимальной доли инвестиции в займе с loan_id = 3
k = 10000 # коэффициент [по условию]
delta = np.round(np.abs(max_piila_in_loan_id_3 - mean_piila) * k, 2)
print(f'Абсолютная величина разности средней доли инвестиции среди всех займов и максимальной доли инвестиции в займе с loan_id = 3: {delta}')

Абсолютная величина разности средней доли инвестиции среди всех займов и максимальной доли инвестиции в займе с loan_id = 3: 1100.0


## **3 задание**

> **Фактические суммы дохода / потерь инвестора**<br>
рассчитываются следующим образом:



```
в качестве дохода берется сумма процентных платежей,
а в качестве потерь - неоплаченный остаток долга в случае, если произошел дефолт.

Относительное значение можно получить путем деления рассчитанной суммы на общую сумму инвестированных в займы средств инвестора.
```



В настоящем задании необходимо рассчитать относительное значение дохода (или потерь) инвесторов *на конец дня 01.03.2023* по каждому.


Для корректности расчета стоит обратить внимание на указанную дату, так как часть дефолтов еще не реализовалась к указанной дате, денежные средства инвесторов еще не были проинвестированы в займы, выданные после этой даты, а процентный доход после указанной даты не был получен инвестором.


В качестве ответа необходимо указать разницу по модулю между максимальным и минимальным значениями относительной доходности инвесторов, умножить на 10 000 и использовать для ответа целую часть.


In [8]:
full_data = pd.read_excel('/content/drive/MyDrive/Crowdlend | Data Analysis Project/Full Inner Data.xlsx')
full_data.head(3)

,Unnamed: 0,investor_id,transaction_date,loan_id,total_amount,principal_payment,interest_payment,loan_rate,raised_amount,term,status,start_date,date_of_default
0,0,3,2022-05-01,592,-11000.0,-11000.0,0.0,0.395,1500000,360,closed,2022-05-01,NaT
1,1,1,2022-05-01,781,-26250.0,-26250.0,0.0,0.395,1500000,180,closed,2022-05-01,NaT
2,2,3,2022-05-01,781,-11000.0,-11000.0,0.0,0.395,1500000,180,closed,2022-05-01,NaT


In [ ]:
# удаляю столбец с индексами Unnamed: 0
full_data.drop(columns='Unnamed: 0', inplace=True)

In [ ]:
# выбираю данные по (включительно) 01.03.2023
data = full_data.query('transaction_date <= "01.01.2023"')
data

,investor_id,transaction_date,loan_id,total_amount,principal_payment,interest_payment,loan_rate,raised_amount,term,status,start_date,date_of_default
0,3,2022-05-01,592,-11000.000000,-11000.000000,0.000000,0.395,1500000,360,closed,2022-05-01,NaT
1,1,2022-05-01,781,-26250.000000,-26250.000000,0.000000,0.395,1500000,180,closed,2022-05-01,NaT
2,3,2022-05-01,781,-11000.000000,-11000.000000,0.000000,0.395,1500000,180,closed,2022-05-01,NaT
3,4,2022-05-01,781,-2500.000000,-2500.000000,0.000000,0.395,1500000,180,closed,2022-05-01,NaT
4,2,2022-05-02,381,-16000.000000,-16000.000000,0.000000,0.206,5000000,360,closed,2022-05-02,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...
6039,4,2023-01-01,808,255.547934,203.711421,51.836513,0.395,500000,360,closed,2022-07-05,NaT
6040,1,2023-01-01,896,4892.630988,4585.766016,306.864972,0.395,2000000,180,closed,2022-08-04,NaT
6041,3,2023-01-01,896,2050.245366,1921.654330,128.591036,0.395,2000000,180,closed,2022-08-04,NaT
6042,3,2023-01-01,970,1124.410909,813.340128,311.070781,0.395,500000,360,closed,2022-10-03,NaT


In [ ]:
# вывожу статусы займов в выбранных данных
data['status'].unique()

array(['closed', 'default'], dtype=object)

буду анализировать профит инвестора по всем закрытым на 01.0.1.2023 займам, а также по дефолтным займам, основная сумма долга по которым была выплачена инвестору

In [ ]:
# рассчитываю прибыль по закрытым займам
info_closed_loans = data.query('status=="closed"')
profit_closed_loans = info_closed_loans.pivot_table(index=['investor_id'], values=['interest_payment'], aggfunc={'interest_payment':'sum'}).rename(columns={'interest_payment':'profit_amount'})
profit_closed_loans

,profit_amount
investor_id,
1,547976.524596
2,231113.679822
3,410506.636425
4,50804.287763
5,156152.569343


In [ ]:
# выбираю данные о займах со статусом default
info_default_loans = data.query('status=="default"')

In [ ]:
pivot_default_loans = info_default_loans.pivot_table(index=['investor_id', 'loan_id'], values=['total_amount'], aggfunc={'total_amount':'sum'})
pivot_default_loans

total_amount
investor_id loan_id              
1           30       -4762.159994
            35      -16464.738024
            48      -26250.000000
            65      -23566.746694
            111     -11572.107036
...                           ...
4           833      -1988.904132
            866      -1477.808264
            924       -407.078094
5           30       -9070.780941
            924      -8141.561881

[85 rows x 1 columns]

In [ ]:
# проверяю наличие займов, по которым была возвращена  сумма займа + % (частично)
pivot_default_loans.query('total_amount >= 0')

,,total_amount
investor_id,loan_id,


> **вывод:**<br>
в рассматриваемых данных не обнаружено займов по которым инвесторам была возвращена сумма займа + проценты (частично)

In [ ]:
# вычисляю суммы выплат по займам для каждого инвестора
loss_by_default_loans = info_default_loans.pivot_table(index=['investor_id'], values=['total_amount'], aggfunc={'total_amount':'sum'})
loss_by_default_loans.rename(columns={'total_amount':'loss_amount'}, inplace=True)
loss_by_default_loans

,loss_amount
investor_id,
1,-555283.108920
2,-30513.249505
3,-89183.887130
4,-46514.012744
5,-17212.342822


> объединяю суммы профита и суммы потерь для оценки успешности стратегии инвесторов

In [ ]:
invested_results = profit_closed_loans.merge(right=loss_by_default_loans, how='inner', on='investor_id')
invested_results['profit/loss'] = np.round((invested_results['profit_amount'] / invested_results['loss_amount']).abs(), 2)
invested_results.sort_values(by='profit/loss', ascending=False)

,profit_amount,loss_amount,profit/loss
investor_id,,,
5,156152.569343,-17212.342822,9.07
2,231113.679822,-30513.249505,7.57
3,410506.636425,-89183.887130,4.60
4,50804.287763,-46514.012744,1.09
1,547976.524596,-555283.108920,0.99


`В качестве ответа необходимо указать разницу по модулю между максимальным и минимальным значениями относительной доходности инвесторов, умножить на 10 000 и использовать для ответа целую часть.`

In [ ]:
range_loss = np.abs(invested_results['profit/loss'].max() - invested_results['profit/loss'].min()) * 1000
range_loss

8080.0